In [1]:
import configparser
#from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
import pyspark.sql.functions as F
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import  StructType, StructField, DoubleType, StringType, IntegerType, TimestampType
import pandas as pd
from datetime import datetime

In [2]:
from pyspark.sql.types import DateType

In [3]:
spark = SparkSession \
.builder \
.appName('Data_lake') \
.getOrCreate()

In [4]:
spark

In [5]:
!ls data

2018-11-01-events.json	2018-11-14-events.json	2018-11-27-events.json
2018-11-02-events.json	2018-11-15-events.json	2018-11-28-events.json
2018-11-03-events.json	2018-11-16-events.json	2018-11-29-events.json
2018-11-04-events.json	2018-11-17-events.json	2018-11-30-events.json
2018-11-05-events.json	2018-11-18-events.json	artists_table
2018-11-06-events.json	2018-11-19-events.json	log-data.zip
2018-11-07-events.json	2018-11-20-events.json	song_data
2018-11-08-events.json	2018-11-21-events.json	song-data.zip
2018-11-09-events.json	2018-11-22-events.json	songplays_table
2018-11-10-events.json	2018-11-23-events.json	songs_table
2018-11-11-events.json	2018-11-24-events.json	time_table
2018-11-12-events.json	2018-11-25-events.json	users_table
2018-11-13-events.json	2018-11-26-events.json


In [7]:
#!unzip data/song-data.zip -d data/
#!unzip data/log-data.zip -d data/

In [8]:
!ls data

2018-11-01-events.json	2018-11-14-events.json	2018-11-27-events.json
2018-11-02-events.json	2018-11-15-events.json	2018-11-28-events.json
2018-11-03-events.json	2018-11-16-events.json	2018-11-29-events.json
2018-11-04-events.json	2018-11-17-events.json	2018-11-30-events.json
2018-11-05-events.json	2018-11-18-events.json	artists_table
2018-11-06-events.json	2018-11-19-events.json	log-data.zip
2018-11-07-events.json	2018-11-20-events.json	song_data
2018-11-08-events.json	2018-11-21-events.json	song-data.zip
2018-11-09-events.json	2018-11-22-events.json	songs_table
2018-11-10-events.json	2018-11-23-events.json	time_table
2018-11-11-events.json	2018-11-24-events.json	users_table
2018-11-12-events.json	2018-11-25-events.json
2018-11-13-events.json	2018-11-26-events.json


In [6]:
song_schema = StructType([
        StructField("artist_id", StringType()),
        StructField("artist_latitude", DoubleType()),
        StructField("artist_location", StringType()),
        StructField("artist_longitude", StringType()),
        StructField("artist_name", StringType()),
        StructField("duration", DoubleType()),
        StructField("num_songs", IntegerType()),
        StructField("title", StringType()),
        StructField("year", IntegerType()),
    ])

In [7]:
df = spark.read.json("data/song_data/A/B/C/*.json",schema=song_schema)

In [8]:
df.show(10)

+------------------+---------------+--------------------+----------------+------------------+---------+---------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|       artist_name| duration|num_songs|               title|year|
+------------------+---------------+--------------------+----------------+------------------+---------+---------+--------------------+----+
|ARNF6401187FB57032|       40.79086|New York, NY [Man...|       -73.96644| Sophie B. Hawkins|  305.162|        1|The Ballad Of Sle...|1994|
|ARLTWXK1187FB5A3F8|       32.74863|      Fort Worth, TX|       -97.32925|       King Curtis|326.00771|        1|A Whiter Shade Of...|   0|
|ARPFHN61187FB575F6|       41.88415|         Chicago, IL|       -87.63241|       Lupe Fiasco|279.97995|        1|Streets On Fire (...|   0|
|AR0IAWL1187B9A96D0|         8.4177|              Panama|       -80.11278|      Danilo Perez|197.19791|        1|         Native Soul|2003|
|ARWB3G61187FB49404|

In [9]:
df.limit(5).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,The Ballad Of Sleeping Beauty,1994
1,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,A Whiter Shade Of Pale (Live @ Fillmore West),0
2,ARPFHN61187FB575F6,41.88415,"Chicago, IL",-87.63241,Lupe Fiasco,279.97995,1,Streets On Fire (Explicit Album Version),0
3,AR0IAWL1187B9A96D0,8.41770,Panama,-80.11278,Danilo Perez,197.19791,1,Native Soul,2003
4,ARWB3G61187FB49404,NaN,"Hamilton, Ohio",None,Steve Morse,363.85914,1,Prognosis,2000


In [12]:
song_data = df.select('song_id', 'title', 'artist_id', 'year', 'duration') \
            .dropDuplicates()

song_data.limit(5).toPandas()

,song_id,title,artist_id,year,duration
0,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.67628
1,SOINLJW12A8C13314C,City Slickers,AR8IEZO1187B99055E,2008,149.86404
2,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
3,SODAUVL12A8C13D184,Prognosis,ARWB3G61187FB49404,2000,363.85914
4,SOLYIBD12A8C135045,Music is what we love,AR051KA1187B98B2FF,0,261.51138


In [13]:
#song_data = song_data.withColumn('id',F.monotonically_increasing_id())

In [14]:
song_data.limit(5).toPandas()

,song_id,title,artist_id,year,duration
0,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.67628
1,SOINLJW12A8C13314C,City Slickers,AR8IEZO1187B99055E,2008,149.86404
2,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,ARJIE2Y1187B994AB7,0,152.92036
3,SODAUVL12A8C13D184,Prognosis,ARWB3G61187FB49404,2000,363.85914
4,SOLYIBD12A8C135045,Music is what we love,AR051KA1187B98B2FF,0,261.51138


In [15]:
songs_table = song_data.write.parquet(path = 'data/songs_table' ,partitionBy = ['year','artist_id'],mode='overwrite')

In [16]:
artists_table = df.selectExpr('artist_id as id', 'artist_name', 'artist_location',         'artist_latitude', 'artist_longitude') \
    .dropDuplicates()
artists_table.limit(10).toPandas()

,id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644
1,AR0IAWL1187B9A96D0,Danilo Perez,Panama,8.41770,-80.11278
2,ARJIE2Y1187B994AB7,Line Renaud,,NaN,NaN
3,AR051KA1187B98B2FF,Wilks,,NaN,NaN
4,ARPFHN61187FB575F6,Lupe Fiasco,"Chicago, IL",41.88415,-87.63241
5,AREVWGE1187B9B890A,Bitter End,Noci (BA),-13.44200,-41.99520
6,ARULZCI1241B9C8611,Luna Orbit Project,,NaN,NaN
7,ARLTWXK1187FB5A3F8,King Curtis,"Fort Worth, TX",32.74863,-97.32925
8,AR8IEZO1187B99055E,Marc Shaiman,,NaN,NaN
9,ARWB3G61187FB49404,Steve Morse,"Hamilton, Ohio",NaN,NaN


In [17]:
artists_table = artists_table.write.parquet(path='data/artists_table',mode = 'overwrite')

In [18]:
df2 = spark.read.json('data/*.json')
df2.show(10)

+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|     artist|      auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+-----------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|   Harmonia| Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy| Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12| 

In [19]:
df2.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [20]:
df2.select('page').dropDuplicates().show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [21]:
df2 = df2.filter("page = 'NextSong'")

In [28]:
get_timestamp = udf(lambda x: int(int(x)/1000),IntegerType())

In [29]:
df2 = df2.withColumn('TimeSatamp', get_timestamp(df2.ts))

In [30]:
get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000)))
df2 = df2.withColumn('start_time', get_datetime(df2.ts))

In [38]:
time_table.limit(10).toPandas()

,start_time,hour,day,week,month,year,weekday
0,2018-11-15 16:14:28.796000,16,15,46,11,2018,5
1,2018-11-15 19:46:21.796000,19,15,46,11,2018,5
2,2018-11-15 22:03:36.796000,22,15,46,11,2018,5
3,2018-11-14 08:10:16.796000,8,14,46,11,2018,4
4,2018-11-14 09:26:21.796000,9,14,46,11,2018,4
5,2018-11-28 21:12:51.796000,21,28,48,11,2018,4
6,2018-11-28 23:08:15.796000,23,28,48,11,2018,4
7,2018-11-05 15:48:00.796000,15,5,45,11,2018,2
8,2018-11-05 18:01:19.796000,18,5,45,11,2018,2
9,2018-11-05 18:09:25.796000,18,5,45,11,2018,2


In [37]:
time_table =  df2.select('start_time')\
                           .withColumn('hour', F.hour('start_time')) \
                           .withColumn('day', F.dayofmonth('start_time')) \
                           .withColumn('week', F.weekofyear('start_time')) \
                           .withColumn('month', F.month('start_time')) \
                           .withColumn('year', F.year('start_time')) \
                           .withColumn('weekday', F.dayofweek('start_time')) \
                           .dropDuplicates()

In [29]:
time_table.write.parquet(path='data/time_table',partitionBy = ['year','month'],mode = 'overwrite')

In [30]:
df.columns


['artist_id',
 'artist_latitude',
 'artist_location',
 'artist_longitude',
 'artist_name',
 'duration',
 'num_songs',
 'song_id',
 'title',
 'year']

In [40]:
df2.columns


['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId',
 'TimeSatamp',
 'start_time']

In [41]:
song = df.select('song_id','artist_id','artist_name')

song.columns

['song_id', 'artist_id', 'artist_name']

In [42]:
log = df2.selectExpr('start_time','userId','level','sessionId','location','userAgent','artist as artist_name')
log.columns


['start_time',
 'userId',
 'level',
 'sessionId',
 'location',
 'userAgent',
 'artist_name']

In [43]:
song.limit(15).toPandas()

,song_id,artist_id,artist_name
0,SONWXQJ12A8C134D94,ARNF6401187FB57032,Sophie B. Hawkins
1,SODREIN12A58A7F2E5,ARLTWXK1187FB5A3F8,King Curtis
2,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,Lupe Fiasco
3,SONSKXP12A8C13A2C9,AR0IAWL1187B9A96D0,Danilo Perez
4,SODAUVL12A8C13D184,ARWB3G61187FB49404,Steve Morse
5,SOFCHDR12AB01866EF,AREVWGE1187B9B890A,Bitter End
6,SOSWKAV12AB018FC91,ARULZCI1241B9C8611,Luna Orbit Project
7,SOUPIRU12A6D4FA1E1,ARJIE2Y1187B994AB7,Line Renaud
8,SORRZGD12A6310DBC3,ARVBRGZ1187FB4675A,Gwen Stefani
9,SOINLJW12A8C13314C,AR8IEZO1187B99055E,Marc Shaiman


In [50]:
log.limit(15).toPandas()

,start_time,userId,level,sessionId,location,userAgent,artist_name
0,2018-11-15 00:30:26.796000,26,free,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",Harmonia
1,2018-11-15 00:41:21.796000,26,free,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",The Prodigy
2,2018-11-15 00:45:41.796000,26,free,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",Train
3,2018-11-15 03:44:09.796000,61,free,597,"Houston-The Woodlands-Sugar Land, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Sony Wonder
4,2018-11-15 05:48:55.796000,80,paid,602,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Van Halen
5,2018-11-15 05:53:44.796000,80,paid,602,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Magic Sam
6,2018-11-15 05:55:56.796000,80,paid,602,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Edward Sharpe & The Magnetic Zeros
7,2018-11-15 06:01:02.796000,80,paid,602,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Usher featuring will.i.am
8,2018-11-15 06:07:37.796000,80,paid,602,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Helen Reddy
9,2018-11-15 06:10:33.796000,80,paid,602,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Taylor Swift


In [45]:
full = log.join(song,on='artist_name',how='inner')

In [60]:
full = full.withColumn('id',F.monotonically_increasing_id())\
.withColumn('month',F.month('start_time'))\
.withColumn('year',F.year('start_time'))

In [49]:
full = full.withColumn('month',F.month('start_time'))

In [ ]:
#song_data = song_data.withColumn('id',F.monotonically_increasing_id())

In [62]:
full.write.parquet(path='data/songplays_table',partitionBy=['year' ,'month'],mode='overwrite')

In [61]:
full.limit(15).toPandas()

,artist_name,start_time,userId,level,sessionId,location,userAgent,song_id,artist_id,id,month,year
0,Lupe Fiasco,2018-11-14 13:11:26.796000,34,free,495,"Milwaukee-Waukesha-West Allis, WI",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,0,11,2018
1,Gwen Stefani,2018-11-14 20:16:39.796000,101,free,603,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",SORRZGD12A6310DBC3,ARVBRGZ1187FB4675A,1,11,2018
2,Lupe Fiasco,2018-11-28 23:22:57.796000,24,paid,984,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,2,11,2018
3,Sophie B. Hawkins,2018-11-24 03:48:21.796000,88,paid,888,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",SONWXQJ12A8C134D94,ARNF6401187FB57032,3,11,2018
4,Lupe Fiasco,2018-11-06 16:04:44.796000,2,free,126,"Plymouth, IN","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,4,11,2018
5,Lupe Fiasco,2018-11-06 18:34:31.796000,97,paid,293,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,5,11,2018
6,Line Renaud,2018-11-22 20:27:18.796000,6,free,763,"Atlanta-Sandy Springs-Roswell, GA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,SOUPIRU12A6D4FA1E1,ARJIE2Y1187B994AB7,6,11,2018
7,Lupe Fiasco,2018-11-10 07:47:51.796000,44,paid,350,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,SOWQTQZ12A58A7B63E,ARPFHN61187FB575F6,7,11,2018


In [117]:
full.columns

['userId',
 'level',
 'sessionId',
 'location',
 'userAgent',
 'artist_id',
 'song_id',
 'artist_id']